In [2]:
pip install -r requirements.txt

  Using cached numpy-1.24.4.tar.gz (10.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "c:\Program Files\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Program Files\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Program Files\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "c:\Program Files\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 77, in _build_backend
          obj = 

In [1]:
import json
import subprocess
import os

def download_10k_filings(ticker, start_year):
    """
    Downloads and extracts 10-K filings for the specified ticker and starting year until the current year.
    This function will clone the EDGAR crawler repository, download the requirements.txt if it's not present,
    set up configurations, and run the download and extract scripts.

    Args:
        ticker (str): The stock ticker symbol for the company (e.g., "AAPL").
        start_year (int): The year from which to start downloading filings.
    """
    # Get the current year
    from datetime import datetime
    current_year = datetime.now().year

    # Clone the repository
    repo_url = "https://github.com/nlpaueb/edgar-crawler.git"
    repo_dir = "edgar-crawler"

    # Clone the repository only if it doesn't exist
    if not os.path.exists(repo_dir):
        print(f"Cloning the repository from {repo_url}...")
        subprocess.run(["git", "clone", repo_url], check=True)

    # Navigate to the repository directory
    os.chdir(repo_dir)

    # Download the requirements.txt only if it doesn't exist
    requirements_file = "requirements.txt"
    if not os.path.exists(requirements_file):
        print(f"Downloading {requirements_file}...")
        subprocess.run(["curl", "-O", "https://raw.githubusercontent.com/nlpaueb/edgar-crawler/master/requirements.txt"], check=True)

    # Install the required dependencies if not already installed
    print("Installing required dependencies...")
    #subprocess.run(["pip", "install", "-r", requirements_file], check=True)

    # Create the configuration dictionary
    config = {
        "download_filings": {
            "start_year": start_year,
            "end_year": current_year,
            "quarters": [1, 2, 3, 4],
            "filing_types": ["10-K"],
            "cik_tickers": [ticker],  # Dynamic ticker
            "user_agent": "Your Name (your-email@example.com)",  # Update with your information
            "raw_filings_folder": "RAW_FILINGS",
            "indices_folder": "INDICES",
            "filings_metadata_file": "FILINGS_METADATA.csv",
            "skip_present_indices": True
        },
        "extract_items": {
            "raw_filings_folder": "RAW_FILINGS",
            "extracted_filings_folder": "EXTRACTED_FILINGS",
            "filings_metadata_file": "FILINGS_METADATA.csv",
            "filing_types": ["10-K"],
            "include_signature": False,
            "items_to_extract": [],
            "remove_tables": True,
            "skip_extracted_filings": True
        }
    }

    # Write the config to a file
    with open('config.json', 'w') as f:
        json.dump(config, f, indent=4)

    # Run the download and extract scripts
    try:
        print(f"Downloading filings for {ticker} from {start_year} to {current_year}...")
        subprocess.run(["python", "download_filings.py"], check=True)

        print(f"Extracting items from filings for {ticker}...")
        subprocess.run(["python", "extract_items.py"], check=True)

        print("Process completed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")
    finally:
        # Navigate back to the original directory
        os.chdir('..')




# Example usage
ticker = "GOOG"  # Replace with the desired ticker
start_year = 2019  # Replace with the desired starting year
download_10k_filings(ticker,start_year)


Cloning the repository from https://github.com/nlpaueb/edgar-crawler.git...
Installing required dependencies...
Extracting items from filings for GOOG...
Process completed successfully!


In [9]:
import os
import json

def extract_all_json_content(folder_path):
    """
    Extracts all content from JSON files in the specified folder, using only the first three parts of the filename.

    Args:
        folder_path (str): Path to the folder containing the JSON files.

    Returns:
        list: A list of dictionaries containing the content of each JSON file.
    """
    extracted_content = []

    # Ensure the folder exists
    if not os.path.exists(folder_path):
        print(f"Error: The folder '{folder_path}' does not exist.")
        return extracted_content

    # Iterate through all files in the folder
    for file_name in os.listdir(folder_path):
        # Process only JSON files
        if file_name.endswith(".json"):
            file_path = os.path.join(folder_path, file_name)

            try:
                # Extract the first three components from the filename
                parts = file_name.replace(".json", "").split("_")[:3]
                if len(parts) < 3:
                    print(f"Skipping invalid filename: {file_name}")
                    continue

                cik, filing_type, year = parts

                # Load the JSON content
                with open(file_path, 'r',encoding="utf-8",errors="ignore") as f:
                    content = json.load(f)

                # Add metadata to the content
                content["cik"] = cik
                content["filing_type"] = filing_type
                content["year"] = year

                # Append the content to the list
                extracted_content.append(content)
                print(f"Successfully extracted data from {file_name}")
            except Exception as e:
                print(f"Error reading {file_name}: {e}")

    return extracted_content


# Example usage
data = extract_all_json_content(r"C:\Users\varun\Documents\SIGMA\NLP Financial Analysis\finstreamlit\edgar-crawler\datasets\EXTRACTED_FILINGS\10-K")

# Print summary of extracted content
print(f"\nExtracted data from {len(data)} files.")
for i, record in enumerate(data[:3]):  # Print a preview of the first 3 records
    print(f"\nRecord {i + 1}:")
    print(json.dumps(record, indent=4))


Successfully extracted data from 1652044_10K_2018_0001652044-19-000004.json
Successfully extracted data from 1652044_10K_2019_0001652044-20-000008.json
Successfully extracted data from 1652044_10K_2020_0001652044-21-000010.json
Successfully extracted data from 1652044_10K_2021_0001652044-22-000019.json
Successfully extracted data from 1652044_10K_2022_0001652044-23-000016.json
Successfully extracted data from 1652044_10K_2023_0001652044-24-000022.json

Extracted data from 6 files.

Record 1:
{
    "cik": "1652044",
    "company": "Alphabet Inc.",
    "filing_type": "10K",
    "filing_date": "2019-02-05",
    "period_of_report": "2018-12-31",
    "sic": "7370",
    "state_of_inc": "DE",
    "state_location": "CA",
    "fiscal_year_end": "1231",
    "filing_html_index": "https://www.sec.gov/Archives/edgar/data/1652044/0001652044-19-000004-index.html",
    "htm_filing_link": "https://www.sec.gov/Archives/edgar/data/1652044/000165204419000004/goog10-kq42018.htm",
    "complete_text_filing_

In [10]:
data[0].keys()

dict_keys(['cik', 'company', 'filing_type', 'filing_date', 'period_of_report', 'sic', 'state_of_inc', 'state_location', 'fiscal_year_end', 'filing_html_index', 'htm_filing_link', 'complete_text_filing_link', 'filename', 'item_1', 'item_1A', 'item_1B', 'item_1C', 'item_2', 'item_3', 'item_4', 'item_5', 'item_6', 'item_7', 'item_7A', 'item_8', 'item_9', 'item_9A', 'item_9B', 'item_9C', 'item_10', 'item_11', 'item_12', 'item_13', 'item_14', 'item_15', 'item_16', 'year'])

In [11]:
compname=data[0]["company"]

In [12]:
compname

'Alphabet Inc.'

In [13]:
data[0]["item_2"]

'ITEM 2.\nPROPERTIES\nOur headquarters are located in Mountain View, California. We also own and lease office and building space in the surrounding areas near our headquarters, which in the aggregate (including our headquarters) represent approximately 11.2 million square feet of office/building space and approximately fifty-nine acres of developable land to accommodate anticipated future growth. In addition, we own and lease office/building space and research and development sites around the world, primarily in North America, Europe, South America, and Asia. We own and operate data centers in the U.S., Europe, South America, and Asia. We believe our existing facilities, both owned and leased, are in good condition and suitable for the conduct of our business.'

In [14]:
data[0]["filing_date"]

'2019-02-05'

In [15]:
result_string=""

In [16]:

result_string = " ".join(data[4].values())

In [18]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

# Load FinBERT model and tokenizer
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def analyze_sentiment_with_rating(text):
    inputs = tokenizer(text, max_length=512, truncation=True, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits.detach().numpy()[0]
    exp_logits = np.exp(logits)
    probabilities = exp_logits / np.sum(exp_logits)

    # Assign a rating based on probabilities
    # Weighted average for the rating: [Neutral, Positive, Negative] → [3, 5, 1]
    rating_weights = [3, 5, 1]
    rating = sum(prob * weight for prob, weight in zip(probabilities, rating_weights))

    return rating, probabilities

# List of dictionaries containing 10-K report data
data=data

# Initialize a dictionary to hold DataFrames for each company
company_dfs = {}

# Process each dictionary in the list
for report in data:
    company_name = report['company']
    report_year = report['year']

    if company_name not in company_dfs:
        # Initialize a DataFrame for the company if it doesn't exist
        columns = [f'item_{i}' for i in range(1, 17)]
        company_dfs[company_name] = pd.DataFrame(columns=['year'] + columns)

    row = {'year': report_year}

    for item in columns:
        if item in report:
            text = report[item]
            rating, _ = analyze_sentiment_with_rating(text)
            row[item] = rating
        else:
            row[item] = None  # Handle missing items

    company_dfs[company_name] = pd.concat([company_dfs[company_name], pd.DataFrame([row])], ignore_index=True)
# Display the results as tables for each company
for company, df in company_dfs.items():
    print(f"Sentiment Scores for {company}:")
    print(df)
    # Optionally, save the results to CSV files
    df.to_csv(f'{company}_10k_report_sentiment_scores.csv', index=False)
    print("\n")


c:\Program Files\Python312\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\varun\AppData\Local\Temp\ipykernel_46268\1707175883.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  company_dfs[company_name] = pd.concat([company_dfs[company_name], pd.DataFrame([row])], ignore_index=True)


Sentiment Scores for Alphabet Inc.:
   year    item_1    item_2    item_3    item_4    item_5    item_6    item_7  \
0  2018  3.193768  2.999986  2.859544  2.916617  3.000020  3.000088  3.723904   
1  2019  4.987228  3.008968  2.853617  2.916617  3.000019  3.000088  3.034284   
2  2020  4.940339  3.008968  2.692213  2.916617  3.000019  3.000088  3.055422   
3  2021  4.943850  3.003889  2.716502  2.916617  3.000013  2.794634  3.132443   
4  2022  4.994903  3.023447  2.716502  2.846894  3.000016  2.794634  4.498439   
5  2023  4.999897  3.036171  2.716502  2.846894  3.000013  2.961737  3.685590   

     item_8  item_9   item_10   item_11   item_12   item_13   item_14  \
0  3.000018  1.9892  2.999562  2.999800  2.999860  2.999942  2.999756   
1  3.000017  1.9892  2.999915  2.999822  2.999699  2.999945  2.999754   
2  3.000000  1.9892  2.999701  2.999803  2.999682  2.999960  2.999751   
3  3.000010  1.9892  2.999517  2.999784  2.999680  2.999930  2.999748   
4  3.000011  1.9892  2.999486  

In [19]:
for company, df in company_dfs.items():
    print(f"\n{company}'s Sentiment Scores:")
    display(df)


Alphabet Inc.'s Sentiment Scores:


,year,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10,item_11,item_12,item_13,item_14,item_15,item_16
0,2018,3.193768,2.999986,2.859544,2.916617,3.000020,3.000088,3.723904,3.000018,1.9892,2.999562,2.999800,2.999860,2.999942,2.999756,3.000000,2.943011
1,2019,4.987228,3.008968,2.853617,2.916617,3.000019,3.000088,3.034284,3.000017,1.9892,2.999915,2.999822,2.999699,2.999945,2.999754,3.000000,2.943011
2,2020,4.940339,3.008968,2.692213,2.916617,3.000019,3.000088,3.055422,3.000000,1.9892,2.999701,2.999803,2.999682,2.999960,2.999751,3.000001,2.943011
3,2021,4.943850,3.003889,2.716502,2.916617,3.000013,2.794634,3.132443,3.000010,1.9892,2.999517,2.999784,2.999680,2.999930,2.999748,3.000001,2.943011
4,2022,4.994903,3.023447,2.716502,2.846894,3.000016,2.794634,4.498439,3.000011,1.9892,2.999486,2.999809,2.999714,2.999930,2.999757,3.000001,2.943011
5,2023,4.999897,3.036171,2.716502,2.846894,3.000013,2.961737,3.685590,3.000021,1.9892,2.999564,2.999859,2.999765,2.999945,2.999889,3.000001,2.992582


In [ ]:
import os

file_name = ""  # Replace with your file name

for root, dirs, files in os.walk("/"):  # Searches from the root directory
    if file_name in files:
        print("File found at:", os.path.join(root, file_name))
        break

In [21]:
df = company_dfs["Alphabet Inc."]

# Compute the column-wise mean for numerical columns
averages = df.select_dtypes(include=[np.number]).mean()

# Convert the averages to a DataFrame and transpose it
averages_df = pd.DataFrame(averages).transpose()

# Add a label for the averages row
averages_df.index = ['Average']

# Concatenate the original DataFrame with the averages DataFrame
df_with_averages = pd.concat([df, averages_df], ignore_index=False)

# Compute the row-wise mean, ignoring non-numeric columns
df_with_averages['row_average'] = df_with_averages.select_dtypes(include=[np.number]).mean(axis=1)

# Output the final DataFrame
print(df_with_averages)

         year    item_1    item_2    item_3    item_4    item_5    item_6  \
0        2018  3.193768  2.999986  2.859544  2.916617  3.000020  3.000088   
1        2019  4.987228  3.008968  2.853617  2.916617  3.000019  3.000088   
2        2020  4.940339  3.008968  2.692213  2.916617  3.000019  3.000088   
3        2021  4.943850  3.003889  2.716502  2.916617  3.000013  2.794634   
4        2022  4.994903  3.023447  2.716502  2.846894  3.000016  2.794634   
5        2023  4.999897  3.036171  2.716502  2.846894  3.000013  2.961737   
Average   NaN  4.676664  3.013572  2.759147  2.893376  3.000017  2.925212   

           item_7    item_8  item_9   item_10   item_11   item_12   item_13  \
0        3.723904  3.000018  1.9892  2.999562  2.999800  2.999860  2.999942   
1        3.034284  3.000017  1.9892  2.999915  2.999822  2.999699  2.999945   
2        3.055422  3.000000  1.9892  2.999701  2.999803  2.999682  2.999960   
3        3.132443  3.000010  1.9892  2.999517  2.999784  2.999680  

In [22]:
df_with_averages

,year,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10,item_11,item_12,item_13,item_14,item_15,item_16,row_average
0,2018,3.193768,2.999986,2.859544,2.916617,3.000020,3.000088,3.723904,3.000018,1.9892,2.999562,2.999800,2.999860,2.999942,2.999756,3.000000,2.943011,2.976567
1,2019,4.987228,3.008968,2.853617,2.916617,3.000019,3.000088,3.034284,3.000017,1.9892,2.999915,2.999822,2.999699,2.999945,2.999754,3.000000,2.943011,3.045762
2,2020,4.940339,3.008968,2.692213,2.916617,3.000019,3.000088,3.055422,3.000000,1.9892,2.999701,2.999803,2.999682,2.999960,2.999751,3.000001,2.943011,3.034048
3,2021,4.943850,3.003889,2.716502,2.916617,3.000013,2.794634,3.132443,3.000010,1.9892,2.999517,2.999784,2.999680,2.999930,2.999748,3.000001,2.943011,3.027427
4,2022,4.994903,3.023447,2.716502,2.846894,3.000016,2.794634,4.498439,3.000011,1.9892,2.999486,2.999809,2.999714,2.999930,2.999757,3.000001,2.943011,3.112860
5,2023,4.999897,3.036171,2.716502,2.846894,3.000013,2.961737,3.685590,3.000021,1.9892,2.999564,2.999859,2.999765,2.999945,2.999889,3.000001,2.992582,3.076727
Average,NaN,4.676664,3.013572,2.759147,2.893376,3.000017,2.925212,3.521680,3.000013,1.9892,2.999624,2.999813,2.999733,2.999942,2.999776,3.000001,2.951273,3.045565


In [25]:
df_with_averages.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 0 to Average
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   year         6 non-null      object 
 1   item_1       7 non-null      float64
 2   item_2       7 non-null      float64
 3   item_3       7 non-null      float64
 4   item_4       7 non-null      float64
 5   item_5       7 non-null      float64
 6   item_6       7 non-null      float64
 7   item_7       7 non-null      float64
 8   item_8       7 non-null      float64
 9   item_9       7 non-null      float64
 10  item_10      7 non-null      float64
 11  item_11      7 non-null      float64
 12  item_12      7 non-null      float64
 13  item_13      7 non-null      float64
 14  item_14      7 non-null      float64
 15  item_15      7 non-null      float64
 16  item_16      7 non-null      float64
 17  row_average  7 non-null      float64
dtypes: float64(17), object(1)
memory usage: 1.0+ KB


In [26]:
dfplot=company_dfs["Alphabet Inc."]

In [27]:
item_descriptions = {
    "item_1": "Business Overview",
    "item_1A": "Risk Factors",
    "item_1B": "Staff Comments",
    "item_1C": "Mine Safety",
    "item_2": "Company Properties",
    "item_3": "Legal Proceedings",
    "item_4": "Mine Disclosures",
    "item_5": "Equity Market",
    "item_6": "Financial Summary",
    "item_7": "Management Analysis",
    "item_7A": "Market Risks",
    "item_8": "Financial Statements",
    "item_9": "Accountant Changes",
    "item_9A": "Control Procedures",
    "item_9B": "Other Information",
    "item_9C": "Jurisdiction Limits",
    "item_10": "Corporate Governance",
    "item_11": "Executive Pay",
    "item_12": "Shareholder Info",
    "item_13": "Related Parties",
    "item_14": "Audit Fees",
    "item_15": "Supporting Docs",
    "item_16": "10-K Summary"
}

In [28]:
import pandas as pd
import plotly.express as px

df = pd.DataFrame(dfplot)

df_long = df.melt(id_vars='year', var_name='Item', value_name='Score')

# Map items to their descriptions
df_long['Item Description'] = df_long['Item'].map(item_descriptions)

# Create the interactive plot
fig = px.line(df_long, x='year', y='Score', color='Item Description', title='Sentiment Scores Over Years for Different Items')

# Show the plot
fig.show()

In [29]:
df_with_averages.rename(columns={"row_average": "Year_Score"}, inplace=True)

In [30]:
df_with_averages['Year_Score']

0          2.976567
1          3.045762
2          3.034048
3          3.027427
4          3.112860
5          3.076727
Average    3.045565
Name: Year_Score, dtype: float64

In [31]:
import plotly.express as px

# Assuming your DataFrame is named df_with_averages and has a 'Year_Score' column
fig = px.line(
    df_with_averages,
    y="Year_Score",
    title="Line Graph of Year Score",
    labels={"Year_Score": "Score"},  # Label for y-axis
    markers=True  # Add markers to the line
)

# Customize layout
fig.update_layout(
    xaxis_title="Index",  # Default x-axis if no column is specified
    yaxis_title="Year Score",
    template="plotly_white",
    title_font_size=18
)

# Show the plot
fig.show()


In [ ]:
below dis sudarshan code

In [32]:
import re



# Regular expressions for company and person names
company_pattern = r"\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\s+Company\b"
people_pattern = r"([A-Z][a-z]+(?:\s[A-Z]\.)?(?:\s[A-Z][a-z]+)+)"
people_matches = re.findall(people_pattern, result_string)

# Filter people matches to exclude irrelevant capitalized text
# Assuming people names appear with clear separation and context
filtered_people = [name for name in people_matches if len(name.split()) >= 2]

# Deduplicate the results
company_names = set(company_pattern)
people_names = set(filtered_people)

# Print the results
print("Company Name:")
for company in company_names:
    print(company)

print("\nPeople Names:")
for person in sorted(people_names):
    print(person)
companies = list(company_names)
people = list(people_names)
s=''
for i in people:
  s+=i+" "

Company Name:
s
)
Z
\
]
m
b
[
n
C
+
z
A
o
y
-
p
?
a
:
*
(

People Names:
Accounting Estimate
In January
Accounts Receivable
Our
Accumulated Other Comprehensive Income
Acquisitions
Mandiant Acquisition
On September
Administrative
The
Advertising Revenues
We
Alphabet Amended
Alphabet Inc
Alphabet Restricted Stock Unit Agreement
Annual Report
Alphabet Restricted Stock Unit Agreement
Quarterly Report
Although English
Among Alphabet Inc
Annual Meeting
Annual Report
Annual Reports
Antitrust Investigations
On November
Appropriate Design Code
At Alphabet
At Google
Attorneys General
Australian Competition
Available Information
Our
Based Award Activities
The
Based Compensation
For
Biometric Identifier Act
Board Matters
Business Combinations
We
California
February
California Age Appropriate Design Code
California Consumer Privacy Act
California Privacy Rights Act
Capital Expenditures
Capital Expenditures
Our
Capital Stock
Capital Stock
Our Board
Cash Equivalents
Cash Equivalents
We
Cash Flow Hedg

In [33]:
import requests

# TextRazor API Key
API_KEY = "068ac1f6ee66ff8b078bd47d186336f9ae76ce919712ab203f287401"  # Replace with your actual API key

# Input text for NER
text = s

# TextRazor API Endpoint
endpoint = "https://api.textrazor.com/"

# Request Headers
headers = {
    "x-textrazor-key": API_KEY,
    "Content-Type": "application/x-www-form-urlencoded"
}

# Request Data
data = {
    "text": text,
    "extractors": "entities"  # Specify that we want to extract entities
}

# Send the POST request
response = requests.post(endpoint, headers=headers, data=data)

# Check the response
if response.status_code == 200:
    response_data = response.json()
    print("TextRazor NER Results:")

    # Extract entities of type 'Person'
    entities = response_data.get("response", {}).get("entities", [])
    people = [entity["entityId"] for entity in entities if "type" in entity and "Person" in entity["type"]]
    print("People Detected:", people)
else:
    print("Error:", response.status_code, response.text)

TextRazor NER Results:
People Detected: ['Sundar Pichai', 'Larry Page', 'Sergey Brin', 'Chief financial officer', 'Martin Ch', 'Eric E. Schmidt']


In [34]:
import requests

def get_top_news(query, api_key):
    """Fetch top news articles based on the query using the News API.

    Args:
        query (str): Search query.
        api_key (str): Your News API key.

    Returns:
        list: A list of URLs of articles related to the query.
    """
    url = "https://newsapi.org/v2/everything"
    params = {
        'q': query,
        'sortBy': 'relevancy',
        'apiKey': api_key
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        return [article["url"] for article in data.get("articles", [])]
    else:
        print(f"Error: Unable to fetch news for query '{query}'. HTTP Status Code: {response.status_code}")
        return []

if __name__ == "__main__":
    # Replace 'YOUR_API_KEY' with your actual News API key
    API_KEY = "dcc1378340a64ab6b76ba8c6ded498d1"

    # Define the company name
    company= compname
  # Change this to any company name as needed

    # Fetch news articles about the company
    print(f"\nFetching news articles for {company}...\n")
    company_query = f'"{company}"'
    company_urls = get_top_news(company_query, API_KEY)

    # Output the results for the company
    print(f"News articles for {company}:\n")
    if company_urls:
        for url in company_urls:
            print(url)
    else:
        print(f"No news articles found for {company}.")

    # Input a list of names
    print(f"\nFetching news articles for {company} and specific individuals...\n")
    names = people  # Ensure you define the `people` variable with the list of names

    # Dictionary to store URLs for each person
    results = {}

    # Fetch news articles for each person and the company
    for person_name in names:
        query = f'"{person_name}" AND "{company}"'
        urls = get_top_news(query, API_KEY)
        results[person_name] = urls

    # Output the results for individuals
    for person, urls in results.items():
        print(f"\nNews articles for {person} and {company}:\n")
        if urls:
            for url in urls:
                print(url)
        else:
            print(f"No news articles found for {person} and {company}.")




Fetching news articles for Alphabet Inc....

News articles for Alphabet Inc.:

https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_733b4397-c95e-4d40-9ec9-f303f9abb25d
https://finance.yahoo.com/news/iphone-maker-hon-hai-clinches-082535817.html
https://finance.yahoo.com/news/google-pichai-joins-tech-ceos-120000759.html
https://www.yahoo.com/news/microsoft-probing-deepseek-linked-group-025844721.html
https://finance.yahoo.com/news/quantum-computing-stocks-drop-nvidia-005300410.html
https://www.tmz.com/2025/01/03/waymo-driverless-car-drives-passenger-circles-misses-flight/
https://finance.yahoo.com/news/google-wants-indulge-workforce-training-125601322.html
https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_94bedb23-2575-493a-aae6-69883eb8fcb4
https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_50dd96e0-142a-4cc9-96aa-3393691c7e3b
https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_64af1182-abbb-4191-a96d-f660158fcf32
https://c